In [22]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [23]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.reset_index(inplace=True)
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,5,5263,555,1,1,0,1
1,2,2015-07-31,5,6064,625,1,1,0,1
2,3,2015-07-31,5,8314,821,1,1,0,1
3,4,2015-07-31,5,13995,1498,1,1,0,1
4,5,2015-07-31,5,4822,559,1,1,0,1


In [24]:
test_df = DataFrame.from_csv("test.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
test_df.reset_index(inplace=True)
test_df.head()

,Store,Date,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-09-17,1,4,1,1,0,0
1,3,2015-09-17,2,4,1,1,0,0
2,7,2015-09-17,3,4,1,1,0,0
3,8,2015-09-17,4,4,1,1,0,0
4,9,2015-09-17,5,4,1,1,0,0


# "Train" the model: Calculate mean per group over full training set

In [25]:
train_df = train_df[train_df['Sales'] > 0]
train_df['Open'] = train_df['Open'].fillna(1)


In [26]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means

means = calculate_means(train_df)
train_df = pandas.merge(train_df, means, on = ['Store','DayOfWeek','Promo'], how='left', suffixes=["", "_predicted"])
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())


In [27]:
means.head()

,Store,DayOfWeek,Promo,Sales
0,1,1,0,4310.446429
1,1,1,1,5852.708333
2,1,2,0,3897.145161
3,1,2,1,5364.597222
4,1,3,0,3869.590164


### Score the model

In [28]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [29]:
score_df = train_df[['Sales', 'Sales_predicted']].copy()
score_df = score_df[score_df['Sales'] != 0]
score_df['PercentError'] =  (score_df['Sales'] - score_df['Sales_predicted']) / score_df['Sales']
score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
math.sqrt(score_df['PercentErrorSquared'].mean())

0.22978844842293555

## Featurize and save

In [30]:
train_df.set_index(['Date', 'Store'], inplace=True)
test_df.set_index(['Date', 'Store'], inplace=True)


In [31]:
train_df['Sales-prediction_mean_error'] = (train_df['Sales_predicted'] - train_df['Sales']) / train_df['Sales']
train_df['Sales-prediction_mean_error'] = train_df['Sales-prediction_mean_error'].fillna(0) # when sales is 0, sales_predicted is nan.

In [39]:
train_df[['Sales_predicted']].to_csv("train-features-predicted_mean.csv")

In [33]:
test_df.reset_index(inplace=True)

In [34]:

test_df = pandas.merge(test_df, means, on = ['Store','DayOfWeek','Promo'], how='left')

In [36]:
test_df['Sales_predicted'] = test_df['Sales']
del test_df['Sales']

In [37]:
test_df.set_index(['Date', 'Store'], inplace=True)

In [38]:
test_df[['Sales_predicted']].to_csv("test-features-predicted_mean.csv")

In [42]:
test_df

Id  DayOfWeek  Open  Promo StateHoliday  SchoolHoliday  \
Date       Store                                                              
2015-09-17 1          1          4     1      1            0              0   
           3          2          4     1      1            0              0   
           7          3          4     1      1            0              0   
           8          4          4     1      1            0              0   
           9          5          4     1      1            0              0   
           10         6          4     1      1            0              0   
           11         7          4     1      1            0              0   
           12         8          4     1      1            0              0   
           13         9          4     1      1            0              0   
           14        10          4     1      1            0              0   
           15        11          4     1      1            0              0   
           16        12          4     1      1            0              0   
           19        13          4     1      1            0              0   
           20        14          4     1      1            0              0   
           21        15          4     1      1            0              0   
           22        16          4     1      1            0              0   
           23        17          4     1      1            0              0   
           24        18          4     1      1            0              0   
           25        19          4     1      1            0              0   
           27        20          4     1      1            0              0   
           29        21          4     1      1            0              0   
           30        22          4     1      1            0              0   
           31        23          4     1      1            0              0   
           32        24          4     1      1            0              0   
           33        25          4     1      1            0              0   
           35        26          4     1      1            0              0   
           36        27          4     1      1            0              0   
           38        28          4     1      1            0              0   
           39        29          4     1      1            0              0   
           40        30          4     1      1            0              0   
...                 ...        ...   ...    ...          ...            ...   
2015-08-01 1077   41059          6     1      0            0              0   
           1078   41060          6     1      0            0              0   
           1079   41061          6     1      0            0              1   
           1080   41062          6     1      0            0              0   
           1083   41063          6     1      0            0              1   
           1084   41064          6     1      0            0              1   
           1086   41065          6     1      0            0              1   
           1087   41066          6     1      0            0              1   
           1088   41067          6     1      0            0              0   
           1089   41068          6     1      0            0              0   
           1091   41069          6     1      0            0              0   
           1092   41070          6     1      0            0              0   
           1094   41071          6     1      0            0              0   
           1096   41072          6     1      0            0              0   
           1097   41073          6     1      0            0              0   
           1099   41074          6     1      0            0              0   
           1100   41075          6     1      0            0              0   
           1101   41076          6     1      0            0           